In [1]:
from decoder import TransformerDecoder
import numpy as np
import tensorflow as tf
from AdamW import AdamW

2023-10-30 15:24:53.138921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 15:24:54.379250: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-10-30 15:24:54.379269: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: archpc
2023-10-30 15:24:54.379273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: archpc
2023-10-30 15:24:54.379351: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.113.1
2023-10-30 15:24:54.379366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 5

In [2]:
#simple token cheme from https://www.tensorflow.org/text/tutorials/word2vec
def gen_vocab(corpus):
    vocab = {}
    vocab["<start>"] = 0
    vocab["<end>"] = 1
    i = 2
    tokens = corpus.lower().split()
    for token in tokens:
        if token  not in vocab:
            vocab[token] = i
        i+=1
    return vocab



In [3]:
corpus = "hello how are you"

vocab = gen_vocab(corpus)

vocab
inverse_vocab = {index: token for token, index in vocab.items()}
inverse_vocab

{0: '<start>', 1: '<end>', 2: 'hello', 3: 'how', 4: 'are', 5: 'you'}

In [40]:

sentence = "<start> hello how are you <end>"
tokenize = lambda sentence : [vocab[word] for word in sentence.lower().split()]
detokenize = lambda tokenized_text : ' '.join([inverse_vocab[x] for x in tokenized_text])

tokenized_text = tokenize(sentence)
input_text = np.array(tokenized_text[:-1])[np.newaxis,:]
target_text = np.array(tokenized_text[1:])[np.newaxis,:]
target_text
input_text

array([[0, 2, 3, 4, 5]])

In [5]:
def get_loss(labels, logits):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)
    )

In [216]:
model = TransformerDecoder(dim_model=512, heads=8, blocks=1, vocab_size=len(vocab), max_len=5)


get_loss(target_text  ,model(input_text))


<tf.Tensor: shape=(), dtype=float32, numpy=2.1251516>

In [81]:
from tqdm import trange
def train(model, optimizer, input_text, output_text, iterations, get_loss):
    bar = trange(iterations)
    for i in bar:
        with tf.GradientTape() as tape:
            est_labels = model(input_text)
            loss = get_loss(labels=target_text, logits=est_labels)

            grads = tape.gradient(loss, model.trainable_variables)

            grads = [tf.convert_to_tensor(grad) for grad in grads]
    
            optimizer.apply_gradients(grads, model.trainable_variables)



        if i%10 == (10 - 1):
            bar.set_description(f"Step {i}; Loss=> {loss.numpy():0.4f}")
            bar.refresh()
    








In [217]:
optimizer = AdamW()
train(model, optimizer, input_text, target_text, 500, get_loss)

Step 219; Loss=> 0.0000:  44%|████▍     | 221/500 [00:10<00:13, 20.96it/s]

In [215]:
x = np.array(tokenized_text[0:1])[np.newaxis,:]


output = model(x)

detokenize(np.array(tf.math.argmax(output, axis = - 1)[0]))




'hello how are you'

In [56]:
x

array([[0, 2, 3, 4, 5]])

In [10]:
def generate(model, prompt, max_token_gen):
        
    for _ in range(max_token_gen):

        output = model(prompt[:, -5:])

        predicted_tokens = tf.math.argmax(output, axis = - 1)
        first_predicted_token = predicted_tokens[:,0]


        prompt = tf.concat([prompt, first_predicted_token[tf.newaxis, :]], axis=1)


    return prompt

In [11]:
output = model(x)

predicted_tokens = tf.math.argmax(output, axis = - 1)
first_predicted_token = predicted_tokens[:,0]


first_predicted_token


<tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>

generate()